In [2]:
# CNN for playing card classification

# Tensorflow-GPU was causing errors. I found this to be a solution
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options = 
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
# device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

# Imports
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
import os

# Define Directory Paths
base_dir = 'Dataset'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
validation_dir = os.path.join(base_dir, 'val')

# Define dataset for training
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(125, 100), batch_size=32, class_mode='categorical')
validation_generator = train_datagen.flow_from_directory(validation_dir, target_size=(125, 100), batch_size=32, class_mode='categorical')

# Creating Model
#Layers
model = models.Sequential()
model.add(layers.Conv2D(64, (3,3), activation = 'relu', input_shape=(125, 100, 3)))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(52, activation='softmax'))

#Freeze conv base and compile model
model.compile(loss='categorical_crossentropy', optimizer = optimizers.RMSprop(learning_rate=0.001), metrics = ['accuracy'])
# history = model.fit(train_generator, epochs=10, validation_data=validation_generator)

Found 102492 images belonging to 52 classes.
Found 12792 images belonging to 52 classes.


In [2]:
# Evaluate model on test data to get accuracy
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(125, 100), class_mode='categorical')
test_loss, test_acc = model.evaluate(test_generator, steps=25)
predictions = model.predict_classes(test_generator, batch_size=None)
print('test acc: ', test_acc)

Found 12844 images belonging to 52 classes.
25/25 [==============================] - 1s 29ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
test acc:  1.0


In [3]:
# Save model
from tensorflow.keras.models import Sequential, save_model, load_model
model.save("Checkpoints/FinalModel.h5")

In [19]:
# Test an image for classification
import numpy as np
from keras.preprocessing import image

img_width, img_height = 125, 100
img = image.load_img('Test1.jpg', target_size = (img_width, img_height))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)

prediction = model.predict(img, batch_size=1)
classes = prediction.argmax(axis=-1)

label_map = (train_generator.class_indices)

for key, value in label_map.items():
    if classes == value:
        print("Card is: " + key)

Card is: 4D


In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 98, 64)       1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 61, 49, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 59, 47, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 29, 23, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 42688)             0         
_________________________________________________________________
dropout (Dropout)            (None, 42688)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               2